In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
from scipy import stats
import numpy as np
import math

ModuleNotFoundError: No module named 'networkx.algorithms.community'

In [ ]:
grad_rates = pd.read_csv('./data/18_19_Grad_Rate_School.csv')

In [ ]:
demo = pd.read_excel('./data/18_19_Demographics.xlsx')

In [ ]:
finance = pd.read_excel('./data/18_19_Finance.xlsx')

### Exploring Graduation Rates Dataset

In [ ]:
grad_rates

In [ ]:
grad_rates.info()

In [ ]:
grad_rates.rename(columns={'system':'DISTRICT_ID', 'school':'SCHOOL_ID'}, inplace=True)
grad_rates.columns

In [ ]:
grad_rates.subgroup.unique()

In [ ]:
grad_rates = grad_rates.loc[grad_rates.subgroup == 'All Students']

In [ ]:
grad_rates = grad_rates[['DISTRICT_ID','SCHOOL_ID','grad_rate']].copy()

In [ ]:
grad_rates.grad_rate.unique()

In [ ]:
grad_rates = grad_rates.loc[~grad_rates.grad_rate.isin(['*', '**'])]

In [ ]:
grad_rates.grad_rate = grad_rates.grad_rate.astype(float)
grad_rates.DISTRICT_ID = grad_rates.DISTRICT_ID.astype('category')
grad_rates.SCHOOL_ID = grad_rates.SCHOOL_ID.astype('category')

In [ ]:
grad_rates.grad_rate = round(grad_rates.grad_rate / 100,2)

In [ ]:
grad_rates

In [ ]:
grad_rates.info()

#### Summary

The grad_rates dataset contains per-school information pertaining to high school graduations rates in the state of Tennessee from 2018-2019. 

The only information I would need from this table is the overall graduation rate per school for all students. We drop the rest of the rows.

- Discovered two entries without graduation rates. Dropped these.

- Renamed system and school to a name that corresponds with what they actually refer to.

- Changed the grad_rate, system, and school to an appropriate type.

- Converted the grad_rate back to decimal percentage.

### Exploring Demographics Dataset

In [ ]:
demo

In [ ]:
demo = demo[['DISTRICT_ID',
             'SCHOOL_ID',
             'TOTAL', 
             'ECONOMICALLY_DISADVANTAGED',
             'FEMALE','HISPANIC_FEMALE','ASIAN_FEMALE','AFRICAN_AMERICAN_FEMALE', 'WHITE_FEMALE',
             'MALE','AFRICAN_AMERICAN_MALE','WHITE_MALE', 'ASIAN_MALE', 'HISPANIC_MALE', 
             'AFRICAN_AMERICAN',
             'ASIAN', 
             'HISPANIC', 
             'WHITE',
             'STUDENTS_WITH_DISABILITIES',
             'NATIVE_AMERICAN',
             'HAWAIIAN_PACISLD', 
             'HAWAIIAN_PACISLD_FEMALE', 'HAWAIIAN_PACISLD_MALE', 
             'NATIVE_AMERICAN_FEMALE', 'NATIVE_AMERICAN_MALE']].copy()

In [ ]:
demo.info()

In [ ]:
demo.DISTRICT_ID = demo.DISTRICT_ID.astype('category')
demo.SCHOOL_ID = demo.SCHOOL_ID.astype('category')

In [ ]:
demo.loc[:, 'ECONOMICALLY_DISADVANTAGED':] = demo.loc[:, 'ECONOMICALLY_DISADVANTAGED':].div(demo.TOTAL,axis=0)

In [ ]:
demo

In [ ]:
demo.loc[:, 'ECONOMICALLY_DISADVANTAGED':].describe()['min':'max']

In [ ]:
demo.drop(columns=['ASIAN', 'ASIAN_MALE', 'ASIAN_FEMALE','STUDENTS_WITH_DISABILITIES', 'NATIVE_AMERICAN',
       'HAWAIIAN_PACISLD', 'HAWAIIAN_PACISLD_FEMALE', 'HAWAIIAN_PACISLD_MALE',
       'NATIVE_AMERICAN_FEMALE', 'NATIVE_AMERICAN_MALE'],inplace=True)

In [ ]:
demo.info()

#### Summary

The demographics data set contains information on the amount of students per listed demographic group.

- Converted total number of students per group to a percentage % that signifies proprotion to that school's total population.

- Dropped columns that contained very little students. Left with these subgroups: 
        ['ECONOMICALLY_DISADVANTAGED', 'FEMALE', 'HISPANIC_FEMALE',
       'ASIAN_FEMALE', 'AFRICAN_AMERICAN_FEMALE', 'WHITE_FEMALE', 'MALE',
       'AFRICAN_AMERICAN_MALE', 'WHITE_MALE', 'ASIAN_MALE', 'HISPANIC_MALE',
       'AFRICAN_AMERICAN', 'ASIAN', 'HISPANIC', 'WHITE']

-Converted school, district id to more appropriate types



### Exploring Financial Dataset

In [ ]:
finance

In [ ]:
finance.info()

In [ ]:
finance.rename(columns={'District ID':'DISTRICT_ID', 'School ID':'SCHOOL_ID', 'Total School Per Pupil Expenditures': 'Expend_per_pupil'}, inplace=True)
finance.columns

In [ ]:
finance.DISTRICT_ID = finance.DISTRICT_ID.astype('category')
finance.SCHOOL_ID = finance.SCHOOL_ID.astype('category')

In [ ]:
finance = finance.loc[~finance.isna().any(axis=1)].copy()

In [ ]:
finance.Expend_per_pupil = finance.Expend_per_pupil.apply(lambda x: round(x, 2))

In [ ]:
finance

In [ ]:
finance.info()

#### Summary
The finance dataset contains various data about finances. I am only interested in Total expenditure per pupil. I deleted the columns I was not intersted in using Excel.

- Renamed school,district id
- Renamed Total School Per Pupil Expenditure to a simpler/more useable name
- Changed ids to more appropriate types
- Dropped any rows that contained nulls.
- Rounded for nicer looking numbers

### Merge Datasets

In [ ]:
grad_demo = pd.merge(grad_rates,demo, how='inner', on=['SCHOOL_ID', 'DISTRICT_ID'])
grad_demo_fin = pd.merge(grad_demo,finance, how='inner', on=['SCHOOL_ID', 'DISTRICT_ID'])

In [ ]:
grad_demo_fin

In [ ]:
grad_demo_fin.info()

In [ ]:
grad_demo_fin.drop(columns=['SCHOOL_ID', 'DISTRICT_ID'], inplace=True)

#### Summary
Merged the three data sets into one table.

In [ ]:
grad_demo_fin.rename(columns={'HISPANIC_FEMALE': 'H_Female',
                             'AFRICAN_AMERICAN_FEMALE':'AA_FEMALE',
                             'WHITE_FEMALE':'W_FEMALE',
                             'AFRICAN_AMERICAN_MALE': 'AA_MALE',
                             'WHITE_MALE': 'W_MALE',
                             'HISPANIC_MALE': 'H_MALE',
                             'AFRICAN_AMERICAN': 'AA',
                             'HISPANIC': 'H',
                             'WHITE': 'W'}, inplace = True)

In [ ]:
grad_demo_fin = grad_demo_fin.loc[grad_demo_fin[(np.abs(stats.zscore(grad_demo_fin)) < 3).all(axis=1)].index]

In [ ]:
grad_demo_fin = grad_demo_fin[~grad_demo_fin.eq(0, axis=0).any(1)]

In [ ]:
grad_demo_fin.to_csv('./data/grad_demo_fin.csv', index=False)

In [ ]:
profile_report = grad_demo_fin.profile_report(html={'style': {'full_width': True}})
profile_report.to_file("./Data/Grad_Demo_Profile.html")